In [1]:
import os
import numpy as np
import pandas as pd
import librosa
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

In [2]:
base_path = 'D:\sem 7\Bismillah Skripsi\Voice Recog\Data\ASVSPOOF 2019\LA'
protocol_dir = os.path.join(base_path, 'ASVspoof2019_LA_cm_protocols')
train_dir = os.path.join(base_path, 'ASVspoof2019_LA_train', 'flac')
dev_dir = os.path.join(base_path, 'ASVspoof2019_LA_dev', 'flac')
eval_dir = os.path.join(base_path, 'ASVspoof2019_LA_eval', 'flac')

In [3]:
def get_file_path(directory, filename):
    return os.path.join(directory, f'{filename}.flac')

In [4]:
def read_dataset(protocol_path, directory):
    """Reads the dataset from a protocol file and returns a DataFrame."""
    df = pd.read_csv(protocol_path, sep=' ', header=None, names=['speaker_id', 'filename', 'system_id', 'null', 'class_name'])
    df['filepath'] = df['filename'].apply(lambda x: get_file_path(directory, x))
    df.drop('null', axis=1, inplace=True)
    df.dropna(inplace=True)
    return df

def label_to_int(class_name):
    if class_name == 'bonafide':
        return 0
    else:
        return 1

def add_columns(df, subset):
    df['target'] = df['class_name'].apply(label_to_int)
    df['subset'] = subset
    return df

def sample_data(df, n_bonafide, n_spoof):
    bonafide = df[df['class_name'] == 'bonafide'].head(n_bonafide)
    spoof = df[df['class_name'] != 'bonafide'].head(n_spoof)
    return pd.concat([bonafide, spoof])

In [5]:
train_df = read_dataset(os.path.join(protocol_dir, 'ASVspoof2019.LA.cm.train.trn.txt'), train_dir)
dev_df = read_dataset(os.path.join(protocol_dir, 'ASVspoof2019.LA.cm.dev.trl.txt'), dev_dir)
eval_df = read_dataset(os.path.join(protocol_dir, 'ASVspoof2019.LA.cm.eval.trl.txt'), eval_dir)

train_df = add_columns(train_df, 'train')
dev_df = add_columns(dev_df, 'dev')
eval_df = add_columns(eval_df, 'eval')

In [6]:
train_df

,speaker_id,filename,system_id,class_name,filepath,target,subset
0,LA_0079,LA_T_1138215,-,bonafide,D:\sem 7\Bismillah Skripsi\Voice Recog\Data\AS...,0,train
1,LA_0079,LA_T_1271820,-,bonafide,D:\sem 7\Bismillah Skripsi\Voice Recog\Data\AS...,0,train
2,LA_0079,LA_T_1272637,-,bonafide,D:\sem 7\Bismillah Skripsi\Voice Recog\Data\AS...,0,train
3,LA_0079,LA_T_1276960,-,bonafide,D:\sem 7\Bismillah Skripsi\Voice Recog\Data\AS...,0,train
4,LA_0079,LA_T_1341447,-,bonafide,D:\sem 7\Bismillah Skripsi\Voice Recog\Data\AS...,0,train
...,...,...,...,...,...,...,...
25375,LA_0098,LA_T_9717580,-,spoof,D:\sem 7\Bismillah Skripsi\Voice Recog\Data\AS...,1,train
25376,LA_0098,LA_T_9779814,-,spoof,D:\sem 7\Bismillah Skripsi\Voice Recog\Data\AS...,1,train
25377,LA_0098,LA_T_9783312,-,spoof,D:\sem 7\Bismillah Skripsi\Voice Recog\Data\AS...,1,train
25378,LA_0098,LA_T_9839348,-,spoof,D:\sem 7\Bismillah Skripsi\Voice Recog\Data\AS...,1,train


In [7]:
train_df.groupby(by="target").agg({
    'target':'count'
})

,target
target,
0,2580
1,22800


In [8]:
def balance_dataset(df, target_column='target', random_state=42):

    # Split the data into two groups based on the target
    class_0 = df[df[target_column] == 0]
    class_1 = df[df[target_column] == 1]

    # Down-sample the majority class to match the size of the minority class
    if len(class_0) > len(class_1):
        class_majority = class_0
        class_minority = class_1
    else:
        class_majority = class_1
        class_minority = class_0

    class_majority_downsampled = class_majority.sample(n=len(class_minority), random_state=random_state)

    # Combine the two classes into one balanced DataFrame
    balanced_df = pd.concat([class_minority, class_majority_downsampled])

    # Shuffle the data to mix the classes
    balanced_df = balanced_df.sample(frac=1, random_state=random_state).reset_index(drop=True)

    # Display the class distribution after down-sampling
    print("Class distribution after balancing:")
    print(balanced_df[target_column].value_counts())

    return balanced_df

In [9]:
train_df = balance_dataset(train_df, target_column='target', random_state=42)
dev_df = balance_dataset(dev_df, target_column='target', random_state=42)

Class distribution after balancing:
target
0    2580
1    2580
Name: count, dtype: int64
Class distribution after balancing:
target
0    2548
1    2548
Name: count, dtype: int64


In [10]:
def extract_features(df):
    """
    Ekstrak fitur audio dari DataFrame dengan kolom file path dan label, serta menerapkan Min-Max Scaler.

    Parameters:
        df (pd.DataFrame): DataFrame yang berisi informasi file audio dan label.

    Returns:
        pd.DataFrame: DataFrame baru dengan fitur-fitur audio yang telah dinormalisasi dan label.
    """
    all_features = []
    
    # Loop melalui setiap baris dalam DataFrame
    for index, row in df.iterrows():
        try:
            # Load audio
            y, sr = librosa.load(row["filepath"], sr=None)
            
            # Menghitung fitur-fitur untuk seluruh audio
            chroma = librosa.feature.chroma_stft(y=y, sr=sr).mean(axis=1)
            rms = librosa.feature.rms(y=y).mean()
            spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr).mean()
            spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr).mean()
            spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr).mean()
            zcr = librosa.feature.zero_crossing_rate(y).mean()
            mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20).mean(axis=1)
            
            # Menentukan label
            label = 0 if row["target"] == 0 else 1

            # Menyimpan fitur dalam satu row dengan label
            feature_row = [
                *chroma, rms, spectral_centroid, spectral_bandwidth, spectral_rolloff, zcr, *mfccs, label
            ]
            all_features.append(feature_row)
        
        except Exception as e:
            print(f"Error processing file {row['filepath']}: {e}")
    
    # Nama kolom untuk DataFrame
    columns = [f'chroma_{i+1}' for i in range(12)] + ['rms_mean', 'spectral_centroid_mean', 'spectral_bandwidth_mean', 
               'spectral_rolloff_mean', 'zcr_mean'] + [f'mfcc_{i+1}' for i in range(20)] + ['label']
    
    # Mengonversi all_features ke dalam DataFrame
    features_df = pd.DataFrame(all_features, columns=columns)
    
    # Menerapkan Min-Max Scaler
    scaler = MinMaxScaler()
    feature_columns = columns[:-1]  # Kolom fitur tanpa 'label'
    features_df[feature_columns] = scaler.fit_transform(features_df[feature_columns])
    
    return features_df


In [11]:
df_train = extract_features(train_df)
df_dev = extract_features(dev_df)

In [12]:
df_train.head()

,chroma_1,chroma_2,chroma_3,chroma_4,chroma_5,chroma_6,chroma_7,chroma_8,chroma_9,chroma_10,...,mfcc_12,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20,label
0,0.310213,0.366204,0.411840,0.364759,0.406845,0.592303,0.626719,0.703758,0.623316,0.536900,...,0.498121,0.724728,0.224966,0.362779,0.686096,0.716601,0.509241,0.510454,0.486522,0
1,0.546096,0.544771,0.520901,0.466460,0.522491,0.562855,0.523238,0.519729,0.457577,0.516732,...,0.203587,0.545144,0.453434,0.473318,0.638694,0.560467,0.686198,0.683154,0.607351,1
2,0.533950,0.463052,0.484185,0.489187,0.436191,0.471218,0.420611,0.451198,0.499898,0.505464,...,0.337224,0.769807,0.501935,0.445049,0.662259,0.500143,0.506444,0.489524,0.546981,0
3,0.548422,0.338033,0.342184,0.464743,0.638609,0.764420,0.501368,0.419646,0.402581,0.396811,...,0.354474,0.630345,0.529009,0.536014,0.479809,0.449247,0.415648,0.428497,0.269411,0
4,0.371969,0.307901,0.392690,0.515731,0.420739,0.425297,0.406176,0.457040,0.568120,0.516034,...,0.555045,0.683649,0.571511,0.478780,0.485717,0.614020,0.492770,0.624638,0.485798,0


In [13]:

df_train = shuffle(df_train, random_state=123).reset_index(drop=True)
df_dev = shuffle(df_dev, random_state=123).reset_index(drop=True)


In [14]:
df_train.to_csv("train_tabular.csv", index=False)
df_dev.to_csv("val_tabular.csv", index=False)

In [ ]:
#df_test = shuffle(df_test, random_state=123).reset_index(drop=True)


# x_train = df_train.iloc[:,:-1]
# y_train = df_train.iloc[:,-1]

# x_dev = df_dev.iloc[:,:-1]
# y_dev = df_dev.iloc[:,-1]

#x_test = df_test.iloc[:,:-1]
#y_test = df_test.iloc[:,-1]